In [1]:
import re
import unicodedata

import pandas as pd
from bs4 import BeautifulSoup

In [12]:
df = pd.read_parquet(
    "../data/02_intermediate/all_contents_extracted/live-healthy-articles.parquet"
)
df = df[
    [
        "LiveHealthyArticle_Title",
        "LiveHealthyArticle_Content_Body",
        "extracted_content_body",
    ]
]
df_empty = df[df["extracted_content_body"] == ""]

display(df_empty)

,LiveHealthyArticle_Title,LiveHealthyArticle_Content_Body,extracted_content_body
1117,deLIGHTS for Diabetic Patients,"<div class=""ExternalClass29ABDACF219847128F662...",


In [13]:
def clean_text(text: str) -> str:
    """
    Cleans the given text by normalizing Unicode characters,
    replacing problematic characters, and removing multiple whitespace.

    Args:
        text (str): The input text to be cleaned.

    Returns:
        str: The cleaned text with normalized Unicode characters,
            problematic characters replaced, and multiple whitespace
            replaced with a single space.
    """
    # Normalize Unicode characters
    text = unicodedata.normalize("NFKD", text)

    # Use ASCII encoding to handle special symbols e.g. copyright \xa9
    text = text.encode("ascii", "ignore").decode("utf-8")

    # Replace common problematic characters
    text = text.replace("\xa0", " ")  # non-breaking space
    text = text.replace("\u200b", "")  # zero-width space
    text = text.replace("\u2028", "\n")  # line separator
    text = text.replace("\u2029", "\n")  # paragraph separator

    # Replace multiple whitespace with single space
    text = re.sub(r"\s+", " ", text)

    return text.strip()


for ind, html_content in enumerate(df_empty["LiveHealthyArticle_Content_Body"]):
    print(ind)
    soup = BeautifulSoup(html_content, "html.parser")
    content = []
    if soup.div is not None:
        soup.div.unwrap()
    # For texts within div
    for tag in soup.find_all("div"):
        if tag.name == "div":
            content.append(clean_text(tag.text))
    processed_text = "\n".join(content).replace("\n\n", "\n").strip()
    print(processed_text, end="\n\n")

0


